# Module: Main Analysis Types
## Sprint: CLV, Customer Segmentation & RFM
## Part 2: BigQueryCode

## Creating the Registration Cohort

In [ ]:
WITH registration_cohort AS (
  SELECT 
    user_pseudo_id,
    MIN(TIMESTAMP_SECONDS(CAST(event_timestamp / 1000000 AS INT64))) AS registration_date
  FROM 
    `tc-da-1.turing_data_analytics.raw_events`
  WHERE 
    TIMESTAMP_SECONDS(CAST(event_timestamp / 1000000 AS INT64)) BETWEEN TIMESTAMP('2020-11-01') AND TIMESTAMP('2021-01-31')
  GROUP BY 
    user_pseudo_id
),

### Explanation
WITH registration_cohort AS (...) creates a temporary table called registration_cohort.
The user_pseudo_id (which is the recommended user ID) is selected and the earliest registration_date for each user.
The WHERE clause ensures that the table only looks at events between November 1, 2020, and January 31, 2021 in accordance to task requirements.
GROUP BY user_pseudo_id groups the data by each user, to get their first sign-up date.

## Calculating weekly revenue for all users

In [ ]:
weekly_revenue AS (
  SELECT 
    user_pseudo_id,
    TIMESTAMP_TRUNC(TIMESTAMP_SECONDS(CAST(event_timestamp / 1000000 AS INT64)), WEEK) AS week_start,
    SUM(IF(event_name = 'purchase', purchase_revenue_in_usd, 0)) AS weekly_revenue
  FROM 
    `tc-da-1.turing_data_analytics.raw_events`
  WHERE 
    TIMESTAMP_SECONDS(CAST(event_timestamp / 1000000 AS INT64)) BETWEEN TIMESTAMP('2020-11-01') AND TIMESTAMP('2021-01-31')
  GROUP BY 
    user_pseudo_id, week_start
),

### Explanation 
WITH weekly_revenue AS creates a temporary table called weekly_revenue.
TIMESTAMP_TRUNC(..., WEEK) AS week_start: This makes sures the table is looking at activity by week.
SUM(IF(event_name = 'purchase', purchase_revenue_in_usd, 0)) AS weekly_revenue: Adds up all the purchase amounts for each user, but only if the event is a purchase.
GROUP BY user_pseudo_id, week_start: Groups the spending by user and week.

## Calculating Weekly Registrations

In [ ]:
weekly_registrations AS (
  SELECT 
    TIMESTAMP_TRUNC(registration_date, WEEK) AS registration_week_start,
    COUNT(DISTINCT user_pseudo_id) AS weekly_registrations
  FROM 
    registration_cohort
  GROUP BY 
    registration_week_start
),

### Explanation
TIMESTAMP_TRUNC(registration_date, WEEK) AS registration_week_start: This groups the registration dates by week.
COUNT(DISTINCT user_pseudo_id) AS weekly_registrations: Counts the unique users who signed up each week.
GROUP BY registration_week_start: Groups the counts by each week.

## Calculating Weekly Revenue per Registration Cohort

In [ ]:
weekly_revenue_per_cohort AS (
  SELECT
    r.user_pseudo_id,
    r.week_start,
    r.weekly_revenue,
    rc.registration_date,
    TIMESTAMP_TRUNC(rc.registration_date, WEEK) AS registration_week_start,
    DATE_DIFF(DATE(TIMESTAMP_TRUNC(r.week_start, WEEK)), DATE(TIMESTAMP_TRUNC(rc.registration_date, WEEK)), WEEK) AS weeks_after_registration
  FROM
    weekly_revenue r
  JOIN
    registration_cohort rc
  ON
    r.user_pseudo_id = rc.user_pseudo_id
),

### Explanation
Here, in this step, links users’ spending to their registration week.
JOIN registration_cohort rc ON r.user_pseudo_id = rc.user_pseudo_id: Combines the weekly_revenue and registration_cohort tables by matching user IDs.
DATE_DIFF(...) AS weeks_after_registration: Calculates the number of weeks between the registration date and each spending week.

## Aggregating and Calculating Revenue per Registration Cohort

In [ ]:
 cohort_revenue AS (
  SELECT
    r.registration_week_start,
    weeks_after_registration,
    SUM(weekly_revenue) AS total_weekly_revenue,
    wr.weekly_registrations
  FROM
    weekly_revenue_per_cohort r
  JOIN
    weekly_registrations wr
  ON
    r.registration_week_start = wr.registration_week_start
  GROUP BY
    registration_week_start, weeks_after_registration, wr.weekly_registrations
),

### Explanation
JOIN weekly_registrations wr ON r.registration_week_start = wr.registration_week_start: Combines the revenue data with the registration counts based on the week they registered.
SUM(weekly_revenue) AS total_weekly_revenue: Adds up the total revenue for each cohort.
GROUP BY registration_week_start, weeks_after_registration, wr.weekly_registrations: Groups these sums by the registration week and the number of weeks since registration.

## Calculate Revenue per User and Outputing Results

In [ ]:
SELECT
  registration_week_start,
  weeks_after_registration,
  total_weekly_revenue / weekly_registrations AS revenue_per_user,
  weekly_registrations
FROM
  cohort_revenue
ORDER BY
  registration_week_start, weeks_after_registration

### Explanation
total_weekly_revenue / weekly_registrations AS revenue_per_user: Divides the total revenue by the number of users who signed up that week to get the average revenue per user.
ORDER BY registration_week_start, weeks_after_registration: Sorts the results by the registration week and how many weeks have passed since then.

## Full Code:

In [ ]:
WITH registration_cohort AS (
  SELECT 
    user_pseudo_id,
    MIN(TIMESTAMP_SECONDS(CAST(event_timestamp / 1000000 AS INT64))) AS registration_date
  FROM 
    `tc-da-1.turing_data_analytics.raw_events`
  WHERE 
    TIMESTAMP_SECONDS(CAST(event_timestamp / 1000000 AS INT64)) BETWEEN TIMESTAMP('2020-11-01') AND TIMESTAMP('2021-01-31')
  GROUP BY 
    user_pseudo_id
),

-- Calculating Weekly Revenue for All Users
weekly_revenue AS (
  SELECT 
    user_pseudo_id,
    TIMESTAMP_TRUNC(TIMESTAMP_SECONDS(CAST(event_timestamp / 1000000 AS INT64)), WEEK) AS week_start,
    SUM(IF(event_name = 'purchase', purchase_revenue_in_usd, 0)) AS weekly_revenue
  FROM 
    `tc-da-1.turing_data_analytics.raw_events`
  WHERE 
    TIMESTAMP_SECONDS(CAST(event_timestamp / 1000000 AS INT64)) BETWEEN TIMESTAMP('2020-11-01') AND TIMESTAMP('2021-01-31')
  GROUP BY 
    user_pseudo_id, week_start
),

-- Calculating Weekly Registrations
weekly_registrations AS (
  SELECT 
    TIMESTAMP_TRUNC(registration_date, WEEK) AS registration_week_start,
    COUNT(DISTINCT user_pseudo_id) AS weekly_registrations
  FROM 
    registration_cohort
  GROUP BY 
    registration_week_start
),

-- Calculating Weekly Revenue per Registration Cohort
weekly_revenue_per_cohort AS (
  SELECT
    r.user_pseudo_id,
    r.week_start,
    r.weekly_revenue,
    rc.registration_date,
    TIMESTAMP_TRUNC(rc.registration_date, WEEK) AS registration_week_start,
    DATE_DIFF(DATE(TIMESTAMP_TRUNC(r.week_start, WEEK)), DATE(TIMESTAMP_TRUNC(rc.registration_date, WEEK)), WEEK) AS weeks_after_registration
  FROM
    weekly_revenue r
  JOIN
    registration_cohort rc
  ON
    r.user_pseudo_id = rc.user_pseudo_id
),

-- Aggregating and Calculating Revenue per Registration Cohort
cohort_revenue AS (
  SELECT
    r.registration_week_start,
    weeks_after_registration,
    SUM(weekly_revenue) AS total_weekly_revenue,
    wr.weekly_registrations
  FROM
    weekly_revenue_per_cohort r
  JOIN
    weekly_registrations wr
  ON
    r.registration_week_start = wr.registration_week_start
  GROUP BY
    registration_week_start, weeks_after_registration, wr.weekly_registrations
)

-- Calculating Revenue per User and Outputing Results
SELECT
  registration_week_start,
  weeks_after_registration,
  total_weekly_revenue / weekly_registrations AS revenue_per_user,
  weekly_registrations
FROM
  cohort_revenue
ORDER BY
  registration_week_start, weeks_after_registration
